In [ ]:
import dash
from dash import dcc, html
from ibkr_connection import connect_ibkr, disconnect_ibkr
from ib_insync import Index, util
import pandas as pd

# Connect to IBKR
ib = connect_ibkr()

# Fetch Data
def fetch_data(symbol='IR.10YR', exchange='EUREX'):
    contract = Index(symbol, exchange)
    bars = ib.reqHistoricalData(
        contract,
        endDateTime='',
        durationStr='1 Y',
        barSizeSetting='1 day',
        whatToShow='MIDPOINT',
        useRTH=True,
        formatDate=1
    )
    df = util.df(bars)
    return df

# Dash App Layout
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("IBKR Interest Rate Dashboard"),
    dcc.Dropdown(
        id='symbol-dropdown',
        options=[
            {'label': '10-Year Treasury Yield', 'value': 'IR.10YR'},
            {'label': '2-Year Treasury Yield', 'value': 'IR.2YR'}
        ],
        value='IR.10YR'
    ),
    dcc.Graph(id='rate-graph')
])

# Callback to Update Graph
@app.callback(
    dash.dependencies.Output('rate-graph', 'figure'),
    [dash.dependencies.Input('symbol-dropdown', 'value')]
)
def update_graph(selected_symbol):
    data = fetch_data(selected_symbol)
    fig = {
        'data': [
            {'x': data['date'], 'y': data['close'], 'type': 'line', 'name': selected_symbol}
        ],
        'layout': {
            'title': f'{selected_symbol} Interest Rate Data'
        }
    }
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)

# Disconnect after use
disconnect_ibkr(ib)
